In [1]:
!pip -q install pandas numpy networkx plotly py3Dmol

import numpy as np
import pandas as pd
import networkx as nx
import plotly.graph_objects as go


In [2]:
# ---------- NODES ----------
nodes = [
    # Layer 0: Mechanical input / ECM
    {"id":"MECH_LOAD", "label":"Tensile load / strain", "layer":0, "type":"stimulus", "compartment":"ECM"},
    {"id":"ECM_COLLAGEN", "label":"ECM collagen fibrils", "layer":0, "type":"structure", "compartment":"ECM"},

    # Layer 1: Membrane mechanosensors
    {"id":"INTEGRINS", "label":"Integrins (e.g., αVβ3 model)", "layer":1, "type":"mechanoreceptor", "compartment":"membrane"},
    {"id":"PIEZO1", "label":"PIEZO1/2 (mechanosensitive)", "layer":1, "type":"ion_channel", "compartment":"membrane"},
    {"id":"TRPV4", "label":"TRPV4 (mechanosensitive)", "layer":1, "type":"ion_channel", "compartment":"membrane"},

    # Layer 2: Focal adhesion / cytoskeletal coupling
    {"id":"TALIN", "label":"Talin", "layer":2, "type":"adaptor", "compartment":"focal_adhesion"},
    {"id":"VINCULIN", "label":"Vinculin", "layer":2, "type":"adaptor", "compartment":"focal_adhesion"},
    {"id":"PAXILLIN", "label":"Paxillin", "layer":2, "type":"adaptor", "compartment":"focal_adhesion"},
    {"id":"FAK", "label":"FAK / PTK2", "layer":2, "type":"kinase", "compartment":"focal_adhesion"},
    {"id":"SRC", "label":"Src family kinases", "layer":2, "type":"kinase", "compartment":"focal_adhesion"},

    # Layer 3: Core signaling modules
    {"id":"CA2", "label":"Ca2+ influx / signaling", "layer":3, "type":"second_messenger", "compartment":"cytosol"},
    {"id":"CAMK", "label":"CaMKII", "layer":3, "type":"kinase", "compartment":"cytosol"},
    {"id":"CALCINEURIN", "label":"Calcineurin", "layer":3, "type":"phosphatase", "compartment":"cytosol"},
    {"id":"RHOA", "label":"RhoA", "layer":3, "type":"GTPase", "compartment":"cytosol"},
    {"id":"ROCK", "label":"ROCK", "layer":3, "type":"kinase", "compartment":"cytosol"},
    {"id":"MAPK_ERK", "label":"MAPK (ERK arm)", "layer":3, "type":"pathway", "compartment":"cytosol"},
    {"id":"MAPK_P38", "label":"MAPK (p38 arm)", "layer":3, "type":"pathway", "compartment":"cytosol"},
    {"id":"MAPK_JNK", "label":"MAPK (JNK arm)", "layer":3, "type":"pathway", "compartment":"cytosol"},
    {"id":"PI3K_AKT", "label":"PI3K–AKT–mTOR", "layer":3, "type":"pathway", "compartment":"cytosol"},
    {"id":"IKK_NFKB", "label":"IKK → NF-κB module", "layer":3, "type":"pathway", "compartment":"cytosol"},
    {"id":"TGFb_SMAD", "label":"TGF-β → Smad2/3", "layer":3, "type":"pathway", "compartment":"cytosol"},

    # Layer 4: Nuclear mechanotransducers / TFs
    {"id":"YAP_TAZ", "label":"YAP/TAZ", "layer":4, "type":"mechano_TF", "compartment":"nucleus"},
    {"id":"TEAD", "label":"TEAD (YAP/TAZ partner)", "layer":4, "type":"TF", "compartment":"nucleus"},
    {"id":"NFAT", "label":"NFAT", "layer":4, "type":"TF", "compartment":"nucleus"},
    {"id":"NFKB", "label":"NF-κB", "layer":4, "type":"TF", "compartment":"nucleus"},
    {"id":"AP1", "label":"AP-1 (c-Fos/c-Jun)", "layer":4, "type":"TF", "compartment":"nucleus"},
    {"id":"SMAD", "label":"Smad2/3 complex", "layer":4, "type":"TF", "compartment":"nucleus"},

    # Layer 5: Gene programs / outputs
    {"id":"COL1A1", "label":"COL1A1 (collagen I)", "layer":5, "type":"gene_output", "compartment":"nucleus→ECM"},
    {"id":"COL3A1", "label":"COL3A1 (collagen III)", "layer":5, "type":"gene_output", "compartment":"nucleus→ECM"},
    {"id":"LOX", "label":"LOX (cross-linking)", "layer":5, "type":"gene_output", "compartment":"nucleus→ECM"},
    {"id":"DCN_BGN", "label":"Decorin/Biglycan (ECM organization)", "layer":5, "type":"gene_output", "compartment":"nucleus→ECM"},
    {"id":"MMPs", "label":"MMP-1/3/13 (matrix degradation)", "layer":5, "type":"gene_output", "compartment":"nucleus→ECM"},
    {"id":"IL6_COX2", "label":"IL-6 / COX-2 (inflammatory)", "layer":5, "type":"gene_output", "compartment":"nucleus→secreted"},
]

nodes_df = pd.DataFrame(nodes)

# ---------- EDGES ----------
# regime: "physio" (adaptacyjne), "overload" (przeciążenie/uraz), "both"
edges = [
    ("MECH_LOAD","ECM_COLLAGEN","transmits","both"),
    ("ECM_COLLAGEN","INTEGRINS","activates","both"),
    ("MECH_LOAD","PIEZO1","activates","both"),
    ("MECH_LOAD","TRPV4","activates","both"),

    ("INTEGRINS","TALIN","activates","both"),
    ("INTEGRINS","PAXILLIN","activates","both"),
    ("TALIN","VINCULIN","activates","both"),

    ("PAXILLIN","FAK","activates","both"),
    ("FAK","SRC","activates","both"),

    ("PIEZO1","CA2","activates","both"),
    ("TRPV4","CA2","activates","both"),
    ("CA2","CAMK","activates","both"),
    ("CA2","CALCINEURIN","activates","both"),
    ("CALCINEURIN","NFAT","activates","both"),

    ("FAK","MAPK_ERK","activates","both"),
    ("SRC","MAPK_ERK","activates","both"),
    ("FAK","PI3K_AKT","activates","both"),

    ("FAK","RHOA","activates","both"),
    ("RHOA","ROCK","activates","both"),

    # mechanosensitive nuclear entry (tension -> YAP/TAZ)
    ("ROCK","YAP_TAZ","activates","physio"),
    ("YAP_TAZ","TEAD","activates","physio"),

    # stress/injury biased arms
    ("FAK","MAPK_P38","activates","overload"),
    ("FAK","MAPK_JNK","activates","overload"),
    ("MAPK_P38","AP1","activates","overload"),
    ("MAPK_JNK","AP1","activates","overload"),
    ("FAK","IKK_NFKB","activates","overload"),
    ("IKK_NFKB","NFKB","activates","overload"),

    # growth factor mechano-crosstalk
    ("PI3K_AKT","YAP_TAZ","activates","physio"),
    ("MECH_LOAD","TGFb_SMAD","activates","both"),
    ("TGFb_SMAD","SMAD","activates","both"),

    # gene outputs (physio)
    ("TEAD","COL1A1","activates","physio"),
    ("TEAD","LOX","activates","physio"),
    ("SMAD","COL1A1","activates","both"),
    ("SMAD","COL3A1","activates","both"),
    ("TEAD","DCN_BGN","activates","physio"),

    # gene outputs (overload)
    ("NFKB","IL6_COX2","activates","overload"),
    ("AP1","MMPs","activates","overload"),
    ("NFKB","MMPs","activates","overload"),
]

edges_df = pd.DataFrame(edges, columns=["source","target","effect","regime"])

display(nodes_df.head(10))
display(edges_df.head(10))
print("Nodes:", len(nodes_df), "Edges:", len(edges_df))


,id,label,layer,type,compartment
0,MECH_LOAD,Tensile load / strain,0,stimulus,ECM
1,ECM_COLLAGEN,ECM collagen fibrils,0,structure,ECM
2,INTEGRINS,"Integrins (e.g., αVβ3 model)",1,mechanoreceptor,membrane
3,PIEZO1,PIEZO1/2 (mechanosensitive),1,ion_channel,membrane
4,TRPV4,TRPV4 (mechanosensitive),1,ion_channel,membrane
5,TALIN,Talin,2,adaptor,focal_adhesion
6,VINCULIN,Vinculin,2,adaptor,focal_adhesion
7,PAXILLIN,Paxillin,2,adaptor,focal_adhesion
8,FAK,FAK / PTK2,2,kinase,focal_adhesion
9,SRC,Src family kinases,2,kinase,focal_adhesion


,source,target,effect,regime
0,MECH_LOAD,ECM_COLLAGEN,transmits,both
1,ECM_COLLAGEN,INTEGRINS,activates,both
2,MECH_LOAD,PIEZO1,activates,both
3,MECH_LOAD,TRPV4,activates,both
4,INTEGRINS,TALIN,activates,both
5,INTEGRINS,PAXILLIN,activates,both
6,TALIN,VINCULIN,activates,both
7,PAXILLIN,FAK,activates,both
8,FAK,SRC,activates,both
9,PIEZO1,CA2,activates,both


Nodes: 33 Edges: 38


In [3]:
def layered_3d_positions(nodes_df, layer_col="layer", radius=2.2, xgap=3.0, jitter=0.25, seed=7):
    rng = np.random.default_rng(seed)
    pos = {}
    for layer, group in nodes_df.groupby(layer_col, sort=True):
        ids = group["id"].tolist()
        n = len(ids)
        for i, node_id in enumerate(ids):
            ang = 2*np.pi*(i/max(n,1))
            y = radius*np.cos(ang) + rng.normal(0, jitter)
            z = radius*np.sin(ang) + rng.normal(0, jitter)
            x = layer * xgap + rng.normal(0, jitter*0.2)
            pos[node_id] = np.array([x,y,z])
    return pos

pos = layered_3d_positions(nodes_df)


In [4]:
layer_names = {
    0:"ECM / bodziec",
    1:"Błona: mechanosensory",
    2:"Ogniska adhezji",
    3:"Szlaki w cytoplazmie",
    4:"Jądro: TF/mechanosensory",
    5:"Program genowy / efekt"
}

def plot_cascade_3d(nodes_df, edges_df, pos, regime="both", title="ACL mechanotransduction – 3D cascade"):
    # filtruj krawędzie
    if regime in ["physio","overload"]:
        ed = edges_df[edges_df["regime"].isin([regime,"both"])].copy()
    else:
        ed = edges_df.copy()

    # --- edges as line segments with None separators ---
    xe, ye, ze = [], [], []
    for _, r in ed.iterrows():
        a = pos[r["source"]]
        b = pos[r["target"]]
        xe += [a[0], b[0], None]
        ye += [a[1], b[1], None]
        ze += [a[2], b[2], None]

    edge_trace = go.Scatter3d(
        x=xe, y=ye, z=ze,
        mode="lines",
        line=dict(width=3),
        hoverinfo="none",
        name="sygnał"
    )

    # --- nodes ---
    node_x, node_y, node_z, node_text, node_layer = [], [], [], [], []
    for _, r in nodes_df.iterrows():
        p = pos[r["id"]]
        node_x.append(p[0]); node_y.append(p[1]); node_z.append(p[2])
        node_layer.append(r["layer"])
        node_text.append(
            f"<b>{r['label']}</b><br>"
            f"type: {r['type']}<br>"
            f"compartment: {r['compartment']}<br>"
            f"layer: {r['layer']} ({layer_names.get(r['layer'],'')})"
        )

    node_trace = go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode="markers+text",
        text=nodes_df["id"],
        textposition="top center",
        hovertext=node_text,
        hoverinfo="text",
        marker=dict(size=9, color=node_layer, colorscale="Viridis", line=dict(width=1)),
        name="węzły"
    )

    fig = go.Figure(data=[edge_trace, node_trace])
    fig.update_layout(
        title=f"{title} | tryb: {regime}",
        scene=dict(
            xaxis_title="warstwy (ECM → geny)",
            yaxis_title="",
            zaxis_title=""
        ),
        height=750,
        margin=dict(l=0,r=0,t=60,b=0)
    )
    return fig

fig = plot_cascade_3d(nodes_df, edges_df, pos, regime="both")
fig.show()


In [5]:
nodes_df.to_csv("acl_mechanotransduction_nodes.csv", index=False)
edges_df.to_csv("acl_mechanotransduction_edges.csv", index=False)

# Interaktywny HTML z wykresem 3D
fig = plot_cascade_3d(nodes_df, edges_df, pos, regime="both")
fig.write_html("acl_mechanotransduction_3d.html")

print("Zapisano: CSV (nodes/edges) i HTML z wizualizacją 3D.")


Zapisano: CSV (nodes/edges) i HTML z wizualizacją 3D.


In [6]:
def plot_acl_schematic_3d(length=10.0, radius=1.0, n_fibers=18, twist=4.0, n_cells=120, seed=1):
    rng = np.random.default_rng(seed)

    # Cylinder surface
    theta = np.linspace(0, 2*np.pi, 60)
    z = np.linspace(0, length, 80)
    theta_grid, z_grid = np.meshgrid(theta, z)
    x_grid = radius * np.cos(theta_grid)
    y_grid = radius * np.sin(theta_grid)

    fig = go.Figure()

    fig.add_trace(go.Surface(
        x=x_grid, y=y_grid, z=z_grid,
        opacity=0.25,
        showscale=False,
        name="ACL body"
    ))

    # Helical fibers
    zz = np.linspace(0, length, 200)
    for k in range(n_fibers):
        phase = 2*np.pi*k/n_fibers
        th = twist * (zz/length) * 2*np.pi + phase
        r_f = radius*0.88
        x = r_f*np.cos(th)
        y = r_f*np.sin(th)
        fig.add_trace(go.Scatter3d(
            x=x, y=y, z=zz,
            mode="lines",
            line=dict(width=4),
            name="collagen fiber" if k==0 else None,
            showlegend=(k==0)
        ))

    # "Cells" as points inside
    cell_z = rng.uniform(0.5, length-0.5, n_cells)
    cell_r = radius * np.sqrt(rng.uniform(0, 1, n_cells)) * 0.75
    cell_t = rng.uniform(0, 2*np.pi, n_cells)
    cell_x = cell_r*np.cos(cell_t)
    cell_y = cell_r*np.sin(cell_t)

    fig.add_trace(go.Scatter3d(
        x=cell_x, y=cell_y, z=cell_z,
        mode="markers",
        marker=dict(size=4),
        name="ligament fibroblasts (schematic)"
    ))

    fig.update_layout(
        title="ACL – 3D schematic (collagen fibers + cells)",
        scene=dict(
            xaxis_title="x",
            yaxis_title="y",
            zaxis_title="long axis (z)"
        ),
        height=800,
        margin=dict(l=0,r=0,t=60,b=0)
    )
    return fig

plot_acl_schematic_3d().show()


In [7]:
import py3Dmol
import requests

def show_pdb(pdb_id, style="cartoon", width=800, height=500):
    pdb_id = pdb_id.lower()
    url = f"https://files.rcsb.org/download/{pdb_id.upper()}.pdb"
    pdb_txt = requests.get(url).text

    view = py3Dmol.view(width=width, height=height)
    view.addModel(pdb_txt, "pdb")
    if style == "cartoon":
        view.setStyle({"cartoon":{}})
    elif style == "stick":
        view.setStyle({"stick":{}})
    else:
        view.setStyle({style:{}})

    view.zoomTo()
    return view

# Przykłady:
show_pdb("1MP8", style="cartoon")


3Dmol.js failed to load for some reason. Please check your browser console for error messages.